In [1]:
import pandas as pd
import os
from pathlib import Path
from unidecode import unidecode
import importlib
import toolkitETL
importlib.reload(toolkitETL)


<module 'toolkitETL' from 'd:\\Projetos\\ETL-ML-Dengue\\src\\etl\\toolkitETL.py'>

In [2]:
diretorio_src_etl = os.getcwd()
diretorio_src = Path(diretorio_src_etl).parent.absolute()
diretorio_base = Path(diretorio_src).parent.absolute()
diretorio_data = os.path.join(diretorio_base, 'data')


In [3]:
diretorio_matriz_distancia_road = os.path.join(diretorio_data, "dist_road_brasil.csv")
diretorio_matriz_distancia_haversine = os.path.join(diretorio_data, "dist_haversine_municipios.csv")
diretorio_ibge_muncipios = os.path.join(diretorio_data, "RELATORIO_DTB_BRASIL_MUNICIPIO.tsv")
diretorio_dengue_sorotipo_genotipo = os.path.join(diretorio_data, "dengue_brasil_sorotipo_genotipo.tsv")

In [4]:
df_road = pd.read_csv(diretorio_matriz_distancia_road, sep=';')

In [5]:
df_road.head()

,orig,dest,dist,dur
0,1100015,1100023,330534.0,"313,6"
1,1100015,1100031,397600.0,"380,7"
2,1100015,1100049,112805.0,"113,9"
3,1100015,1100056,390938.0,"373,2"
4,1100015,1100064,353606.0,"333,9"


In [8]:
df_haver = pd.read_csv(diretorio_matriz_distancia_haversine, sep=',')

In [9]:
df_haver.head()

,orig,dest,dist
0,5200050,3100104,289.830897
1,5200050,5200100,100.160142
2,5200050,3100203,499.787149
3,5200050,1500107,1672.999956
4,5200050,2300101,1539.696725


In [10]:
df_sorotipo = pd.read_csv(diretorio_dengue_sorotipo_genotipo, sep='\t')
df_sorotipo_filtrado = df_sorotipo[['Virus name', 'Serotype', 'Genotype', 'Accession ID', 'Collection date', 'Location']].copy()

In [11]:
df_sorotipo_filtrado['Estado'] = df_sorotipo_filtrado['Location'].apply(toolkitETL.get_estado)
df_sorotipo_filtrado['Municipio'] = df_sorotipo_filtrado['Location'].apply(toolkitETL.get_municipio)

In [12]:
df_sorotipo_filtrado.head()

,Virus name,Serotype,Genotype,Accession ID,Collection date,Location,Estado,Municipio
0,hDenV2/Brazil/un-FAMERP-2395/2019,DENV2,III-Asian-American,EPI_ISL_11579708,2019-05-28,South America / Brazil,<NA>,<NA>
1,hDenV2/Brazil/un-FAMERP-2411/2019,DENV2,III-Asian-American,EPI_ISL_11579724,2019-05-20,South America / Brazil,<NA>,<NA>
2,hDenV2/Brazil/un-FAMERP-2400/2019,DENV2,III-Asian-American,EPI_ISL_11579725,2019-05-09,South America / Brazil,<NA>,<NA>
3,hDenV2/Brazil/un-FAMERP-2413/2019,DENV2,III-Asian-American,EPI_ISL_11579726,2019-05-09,South America / Brazil,<NA>,<NA>
4,hDenV2/Brazil/un-FAMERP-2397/2019,DENV2,III-Asian-American,EPI_ISL_11579754,2019-05-08,South America / Brazil,<NA>,<NA>


In [13]:
df_ibge_municipios = pd.read_csv(diretorio_ibge_muncipios, sep='\t', encoding='latin1')
df_ibge_municipios_filtrado = df_ibge_municipios[['UF','Nome_UF', 'Código Município Completo', 'Nome_Município']].copy()

In [14]:
df_ibge_municipios_filtrado['Municipio'] = df_ibge_municipios_filtrado['Nome_Município'].apply(lambda x: unidecode(x))
df_ibge_municipios_filtrado['Estado'] = df_ibge_municipios_filtrado['Nome_UF'].apply(lambda x: unidecode(x))

In [15]:
df_ibge_municipios_filtrado.head()

,UF,Nome_UF,Código Município Completo,Nome_Município,Municipio,Estado
0,11,Rondônia,1100015,Alta Floresta D'Oeste,Alta Floresta D'Oeste,Rondonia
1,11,Rondônia,1100379,Alto Alegre dos Parecis,Alto Alegre dos Parecis,Rondonia
2,11,Rondônia,1100403,Alto Paraíso,Alto Paraiso,Rondonia
3,11,Rondônia,1100346,Alvorada D'Oeste,Alvorada D'Oeste,Rondonia
4,11,Rondônia,1100023,Ariquemes,Ariquemes,Rondonia


In [16]:
# Realizar o merge dos DataFrames
df_merged = pd.merge(df_sorotipo_filtrado, df_ibge_municipios_filtrado, on=['Municipio', 'Estado'], how='left')

In [17]:
# Remover as linhas onde 'Codigo Municipio Completo' é NaN
df_merged_clean = df_merged.dropna(subset=['Código Município Completo'])

In [ ]:
# Converter a coluna 'Collection date' para o formato datetime
df_merged_clean['Collection date'] = pd.to_datetime(df_merged_clean['Collection date'])

# Extrair mês e ano usando .loc[] para evitar o aviso SettingWithCopyWarning
df_merged_clean.loc[:, 'Ano'] = df_merged_clean['Collection date'].dt.year
df_merged_clean.loc[:, 'Mes'] = df_merged_clean['Collection date'].dt.month



In [20]:
df_teste = pd.read_csv(os.path.join(diretorio_data,'testes', 'teste.txt'), sep='\t')

# Converter a coluna 'Data' para o formato datetime
df_teste['Data'] = pd.to_datetime(df_teste['Data'])
# df_teste.loc[:, 'Data'] = pd.to_datetime(df_teste['Data'], format="%Y-%m-%d")

# # Extrair mês e ano usando .loc[] para evitar o aviso SettingWithCopyWarning
df_teste.loc[:, 'Ano'] = df_teste['Data'].dt.year
df_teste.loc[:, 'Mes'] = df_teste['Data'].dt.month

df_teste.head()

,Municipio,Data,Ano,Mes
0,3550308,2020-03-15,2020,3
1,3550308,2011-07-22,2011,7
2,2304400,2018-01-13,2018,1
3,2304400,2022-03-01,2022,3


In [23]:
sorotipos_genotipos = {
    'DENV1':['V', 'unassigned', 'nan'],
    'DENV2':['II-Cosmopolitan','III-Asian-American', 'IV-AsianII', 'V', 'unassigned'],
    'DENV3':['III'],
    'DENV4':['II', 'V'],    

}

dicionario_distancias = {
    'haver': df_haver,
    'road': df_road
}

In [24]:
# sorotipos = df_merged_clean['Serotype'].unique() # Fixando o número de genótipos e sorotipos de dengue para manter o número de colunas do modelo

new_columns = {}
for distancia, df_distancia in dicionario_distancias.items():
    for i in range(0, 2):
        for sorotipo in sorotipos_genotipos.keys():
            genotipos = sorotipos_genotipos[sorotipo]
            for genotipo in genotipos:
                if i == 1:
                    nome_coluna = f'menor_dist_{distancia}_{sorotipo}_{genotipo}_menos_{i}_mes'
                else:
                    nome_coluna = f'menor_dist_{distancia}_{sorotipo}_{genotipo}_menos_{i}_meses'
                
                new_columns[nome_coluna] = df_teste.apply(lambda row: toolkitETL.get_distancia_mais_proxima_sorotipo_genotipo(df_distancia, df_merged_clean,
                    row["Ano"], row["Mes"], row["Municipio"], genotipo, sorotipo, subtracao_mes=i), axis=1)


In [25]:
new_columns_df = pd.DataFrame(new_columns)

# Concatenar as novas colunas ao DataFrame original
df_teste = pd.concat([df_teste, new_columns_df], axis=1)

# Para garantir que o DataFrame resultante não seja fragmentado, crie uma cópia
df_teste = df_teste.copy()

In [26]:
df_teste

,Municipio,Data,Ano,Mes,menor_dist_haver_DENV1_V_menos_0_meses,menor_dist_haver_DENV1_unassigned_menos_0_meses,menor_dist_haver_DENV1_nan_menos_0_meses,menor_dist_haver_DENV2_II-Cosmopolitan_menos_0_meses,menor_dist_haver_DENV2_III-Asian-American_menos_0_meses,menor_dist_haver_DENV2_IV-AsianII_menos_0_meses,...,menor_dist_road_DENV1_unassigned_menos_1_mes,menor_dist_road_DENV1_nan_menos_1_mes,menor_dist_road_DENV2_II-Cosmopolitan_menos_1_mes,menor_dist_road_DENV2_III-Asian-American_menos_1_mes,menor_dist_road_DENV2_IV-AsianII_menos_1_mes,menor_dist_road_DENV2_V_menos_1_mes,menor_dist_road_DENV2_unassigned_menos_1_mes,menor_dist_road_DENV3_III_menos_1_mes,menor_dist_road_DENV4_II_menos_1_mes,menor_dist_road_DENV4_V_menos_1_mes
0,3550308,2020-03-15,2020,3,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,3550308,2011-07-22,2011,7,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,2304400,2018-01-13,2018,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,2304400,2022-03-01,2022,3,635.555308,<NA>,<NA>,1857.444425,2268.327179,<NA>,...,<NA>,<NA>,2304987.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [27]:
# df_teste.to_csv(os.path.join(diretorio_data,'testes', 'output_teste.tsv'), sep='\t', index=False)